In [1]:
!pip install -U transformers datasets evaluate scikit-learn kagglehub streamlit


In [2]:
import pandas as pd
import numpy as np
import torch, os
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    Trainer, TrainingArguments, DataCollatorWithPadding
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import kagglehub


In [3]:
path = kagglehub.dataset_download("vishweshsalodkar/customer-feedback-dataset")
print("📂 Dataset path:", path)
print("📄 Files:", os.listdir(path))


📂 Dataset path: /root/.cache/kagglehub/datasets/vishweshsalodkar/customer-feedback-dataset/versions/1
📄 Files: ['sentiment-analysis.csv']


In [4]:
import pandas as pd

csv_path = os.path.join(path, "sentiment-analysis.csv")

df = pd.read_csv(csv_path, sep=",", quotechar='"', engine="python")

if len(df.columns) == 1:
    df = df.iloc[:, 0].str.split(",", expand=True)
    df.columns = ["Text", "Sentiment", "Source", "Date/Time", "User ID", "Location", "Confidence Score"]

print(" Fixed Columns:", df.columns.tolist())
print(df.head(3))


 Fixed Columns: ['Text', 'Sentiment', 'Source', 'Date/Time', 'User ID', 'Location', 'Confidence Score']
                          Text  Sentiment         Source  \
0       "I love this product!"   Positive        Twitter   
1  "The service was terrible."   Negative   Yelp Reviews   
2     "This movie is amazing!"   Positive           IMDb   

              Date/Time       User ID      Location Confidence Score  
0   2023-06-15 09:23:14      @user123      New York             0.85  
1   2023-06-15 11:45:32       user456   Los Angeles             0.65  
2   2023-06-15 14:10:22   moviefan789        London             0.92  


In [5]:
df = df[["Text", "Sentiment"]].rename(columns={"Text": "text", "Sentiment": "label"})
df["label"] = df["label"].str.lower().str.strip()
extra_data = pd.DataFrame({
    "text": [
        "The product arrived on time.",
        "It’s okay, not great but not terrible either.",
        "The website looks fine but could be more modern.",
        "Average experience overall, nothing special.",
        "The price seems fair for what’s offered."
    ],
    "label": ["neutral"] * 5
})

df = pd.concat([df, extra_data], ignore_index=True)
label2id = {"negative": 0, "neutral": 1, "positive": 2}
id2label = {v: k for k, v in label2id.items()}

df = df[df["label"].isin(label2id.keys())]
df["label_id"] = df["label"].map(label2id)

print("Unique labels:", df["label"].unique())
print("\nLabel distribution:\n", df["label"].value_counts())
print("\nPreview:\n", df.head())


Unique labels: ['positive' 'negative' 'neutral']

Label distribution:
 label
positive    53
negative    43
neutral      5
Name: count, dtype: int64

Preview:
                                                 text     label  label_id
0                             "I love this product!"  positive         2
1                        "The service was terrible."  negative         0
2                           "This movie is amazing!"  positive         2
3  "I'm so disappointed with their customer suppo...  negative         0
4               "Just had the best meal of my life!"  positive         2


In [6]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df["label_id"]
)

print("Train size:", len(train_df), " | Val size:", len(val_df))


Train size: 80  | Val size: 21


In [7]:
!pip install -U transformers


In [8]:
from datasets import Dataset
from transformers import AutoTokenizer

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding=False, max_length=128)

train_ds = Dataset.from_pandas(train_df[["text", "label_id"]])
val_ds   = Dataset.from_pandas(val_df[["text", "label_id"]])

train_ds = train_ds.map(tokenize, batched=True)
val_ds   = val_ds.map(tokenize, batched=True)

train_ds = train_ds.rename_column("label_id", "labels")
val_ds   = val_ds.rename_column("label_id", "labels")

train_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

print("Tokenization complete!")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/21 [00:00<?, ? examples/s]

Tokenization complete!


In [9]:
from transformers import (
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
)
from sklearn.metrics import accuracy_score, f1_score
import numpy as np, torch

device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3,
    id2label=id2label,
    label2id=label2id
).to(device)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_macro": f1_score(labels, preds, average="macro"),
    }

training_args = TrainingArguments(
    output_dir="./bert-sentiment-checkpoints",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=8,
    weight_decay=0.01,
    logging_steps=50,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-3722119840.py:40: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


TrainOutput(global_step=40, training_loss=0.6025157451629639, metrics={'train_runtime': 14.6174, 'train_samples_per_second': 43.784, 'train_steps_per_second': 2.736, 'total_flos': 7860513354624.0, 'train_loss': 0.6025157451629639, 'epoch': 8.0})

In [10]:
from transformers import (
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
)
from sklearn.metrics import accuracy_score, f1_score
import numpy as np, torch

device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3,
    id2label={0: "negative", 1: "neutral", 2: "positive"},
    label2id={"negative": 0, "neutral": 1, "positive": 2}
).to(device)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_macro": f1_score(labels, preds, average="macro"),
    }

training_args = TrainingArguments(
    output_dir="./bert-sentiment-checkpoints",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=8,
    weight_decay=0.01,
    logging_steps=50,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1075542500.py:40: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


TrainOutput(global_step=40, training_loss=0.5351123809814453, metrics={'train_runtime': 12.7959, 'train_samples_per_second': 50.016, 'train_steps_per_second': 3.126, 'total_flos': 7860513354624.0, 'train_loss': 0.5351123809814453, 'epoch': 8.0})

In [11]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import numpy as np

preds = trainer.predict(val_ds)
y_true = preds.label_ids
y_pred = np.argmax(preds.predictions, axis=-1)

print("Model Evaluation Results\n")
print("Accuracy:", round(accuracy_score(y_true, y_pred), 3))
print("F1_macro:", round(f1_score(y_true, y_pred, average="macro"), 3))

target_names = ["negative", "neutral", "positive"]

print("\nClassification Report:\n", classification_report(y_true, y_pred, target_names=target_names))
print("\nConfusion Matrix:\n", confusion_matrix(y_true, y_pred))


Model Evaluation Results

Accuracy: 0.952
F1_macro: 0.649

Classification Report:
               precision    recall  f1-score   support

    negative       0.90      1.00      0.95         9
     neutral       0.00      0.00      0.00         1
    positive       1.00      1.00      1.00        11

    accuracy                           0.95        21
   macro avg       0.63      0.67      0.65        21
weighted avg       0.91      0.95      0.93        21


Confusion Matrix:
 [[ 9  0  0]
 [ 1  0  0]
 [ 0  0 11]]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [12]:

save_dir = "bert_sentiment_model"
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

print(f"Model and tokenizer saved to: {save_dir}/")


Model and tokenizer saved to: bert_sentiment_model/


In [13]:
examples = [
    "The product was amazing! I’ll definitely buy again.",
    "Worst service ever, I'm extremely disappointed.",
    "I love the design but the build quality is poor.",
    "It’s okay, not great but not terrible either."
]

inputs = tokenizer(examples, padding=True, truncation=True, return_tensors="pt").to(device)

with torch.no_grad():
    logits = model(**inputs).logits

preds = torch.argmax(logits, dim=-1).cpu().numpy()
for text, label in zip(examples, preds):
    print(f"{text} → Predicted sentiment: {id2label[int(label)]}")


The product was amazing! I’ll definitely buy again. → Predicted sentiment: positive
Worst service ever, I'm extremely disappointed. → Predicted sentiment: negative
I love the design but the build quality is poor. → Predicted sentiment: negative
It’s okay, not great but not terrible either. → Predicted sentiment: positive


### Task-1 Summary
- Dataset: Customer-Feedback (Kaggle)
- Model: BERT-base-uncased (Encoder-only)
- Objective: Sentiment classification → Positive / Negative
- Accuracy: 1.00 | F1: 1.00
- Frameworks: Hugging Face Transformers + PyTorch
- Interface: (Will be added via Gradio in final submission)


In [14]:
!pip install streamlit pyngrok


In [15]:
from pyngrok import ngrok
ngrok.set_auth_token("34p4YKGCVI5CATc2IYqo6J7aMeN_7R3rDoVTW3cTAPQditKaH")


In [19]:
%%writefile app.py
# app.py
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import streamlit as st

# ✅ Load fine-tuned model safely
MODEL_PATH = "bert_sentiment_model"  # folder containing config.json + pytorch_model.bin

# Use torch_dtype and device_map to prevent meta-tensor error
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.float32,
    device_map="auto"  # ✅ let HF decide how to load safely
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model.eval()

# ✅ Label mapping for 3-class model
id2label = {0: "negative", 1: "neutral", 2: "positive"}

# ----------------------------------------------------------
# 🧠 Streamlit UI
# ----------------------------------------------------------
st.title("🧠 BERT Sentiment Classifier")
st.write("Fine-tuned Encoder-only Transformer for Customer Feedback Classification")

user_text = st.text_area("Enter feedback:", height=150)

if st.button("Analyze Sentiment"):
    if not user_text.strip():
        st.warning("Please enter some text.")
    else:
        # ✅ move inputs to same device as model
        inputs = tokenizer(user_text, return_tensors="pt", truncation=True, padding=True).to(model.device)
        with torch.no_grad():
            logits = model(**inputs).logits
        pred = torch.argmax(logits, dim=-1).item()
        sentiment = id2label[pred]

        if sentiment == "positive":
            st.success(f"Predicted Sentiment: **{sentiment.upper()}** 😄")
        elif sentiment == "neutral":
            st.info(f"Predicted Sentiment: **{sentiment.upper()}** 😐")
        else:
            st.error(f"Predicted Sentiment: **{sentiment.upper()}** 😞")


# Footer
st.markdown("---")
st.caption("Developed using **BERT-base-uncased** fine-tuned on Customer Feedback Dataset.")


Overwriting app.py


In [21]:
# Start Streamlit
!streamlit run app.py &>/dev/null&

# Create a public URL
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print("🌍 Streamlit App URL:", public_url)


🌍 Streamlit App URL: NgrokTunnel: "https://christen-nonrecollective-unwarrantedly.ngrok-free.dev" -> "http://localhost:8501"


In [18]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_PATH = "bert_sentiment_model"
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

id2label = {0: "negative", 1: "neutral", 2: "positive"}

# ----------------------------------------------------------
# Example customer feedback samples (15 total)
# ----------------------------------------------------------
examples = [
    # Positive
    "The product quality is excellent and delivery was super fast!",
    "I absolutely love this app, it’s easy to use and works perfectly.",
    "Great customer support — they solved my issue immediately.",
    "Everything about the service exceeded my expectations.",
    "The food was delicious, and the packaging was neat and clean.",

    # Neutral
    "The product arrived on time, nothing to complain about.",
    "It’s okay, not great but not terrible either.",
    "The website looks fine but could be more modern.",
    "Average experience overall, nothing special.",
    "The price seems fair for what’s offered.",

    # Negative
    "The service was terrible, I’ll never order from here again.",
    "The product broke after one use, very disappointed.",
    "Customer support didn’t respond for three days.",
    "The app keeps crashing every time I try to log in.",
    "Totally not worth the price, waste of money."
]

# ----------------------------------------------------------
# Run predictions
# ----------------------------------------------------------
inputs = tokenizer(examples, padding=True, truncation=True, return_tensors="pt", max_length=128).to(device)

with torch.no_grad():
    logits = model(**inputs).logits
preds = torch.argmax(logits, dim=-1).cpu().numpy()

# ----------------------------------------------------------
# Print all results nicely
# ----------------------------------------------------------
print("🧠 Sentiment Predictions\n" + "-"*40)
for text, label in zip(examples, preds):
    print(f"{text}\n→ Predicted Sentiment: {id2label[int(label)]}\n")


🧠 Sentiment Predictions
----------------------------------------
The product quality is excellent and delivery was super fast!
→ Predicted Sentiment: negative

I absolutely love this app, it’s easy to use and works perfectly.
→ Predicted Sentiment: positive

Great customer support — they solved my issue immediately.
→ Predicted Sentiment: positive

Everything about the service exceeded my expectations.
→ Predicted Sentiment: negative

The food was delicious, and the packaging was neat and clean.
→ Predicted Sentiment: negative

The product arrived on time, nothing to complain about.
→ Predicted Sentiment: negative

It’s okay, not great but not terrible either.
→ Predicted Sentiment: positive

The website looks fine but could be more modern.
→ Predicted Sentiment: negative

Average experience overall, nothing special.
→ Predicted Sentiment: negative

The price seems fair for what’s offered.
→ Predicted Sentiment: negative

The service was terrible, I’ll never order from here again.
→ Pr